## Thinking1：KNN与KMeans中的K分别代表什么？
### answer：K-Means 中的 K 值代表 K 类。KNN 中的 K 值代表 K 个最接近的邻居。 

## Thinking2:都有哪些常用的启发式算法？
### answer：常用的算法有：模拟退火算法（SA）、遗传算法（GA）、蚁群算法（ACO）、人工神经网络（ANN)。

## Thinking3:遗传算法的原理是怎样的？
### answer：遗传算法是一类借鉴生物界的进化规律（适者生存，优胜劣汰遗传机制）演化而来的随机化搜索方法，其主要特点是直接对结构对象进行操作，不存在求导和函数连续性的限定，具有内在的隐并行性和更好的全局寻优能力。

In [10]:
#Action 1
import pandas as pd
#数据加载
file = 'jobs_4k.xls'
content = pd.read_excel(file)
print(content.head())
position_names = content['positionName'].tolist()
skill_labels = content['skillLables'].tolist()

#图的可视化
from collections import defaultdict #带有默认值的字典
skill_position_graph = defaultdict(list)
for p, s in zip(position_names, skill_labels):
    skill_position_graph[p] += eval(s)
#print(skill_position_graph)

import networkx as nx
import matplotlib.pyplot as plt
G = nx.Graph(skill_position_graph)
#设置中文字体为黑体
plt.rcParams['font.sans-serif'] = ['SimHei']
#用来正常显示负号
plt.rcParams['axes.unicode_minus'] = False


#随机选取30个工作岗位
import random
sample_nodes = random.sample(position_names, k=30)
print(sample_nodes)
print(len(position_names))
sample_nodes_connections = sample_nodes
for p, skills in skill_position_graph.items():
    if p in sample_nodes:
        sample_nodes_connections += skills
print(sample_nodes_connections)
'''
#抽取原始G中的节点作为子图
sample_graph = G.subgraph(sample_nodes_connections)
plt.figure(figsize=(16, 8))
pos = nx.spring_layout(sample_graph, k=1)
nx.draw(sample_graph, pos, with_labels=True)
plt.show()
'''
#使用Pagerank计算节点（技能）影响力
pr = nx.pagerank(G, alpha=0.9)
ranked_position_and_ability = sorted([(name, value) for name, value in pr.items()], key=lambda x:x[1], reverse=True)
#print(ranked_position_and_ability)

#特征X去掉了salary
X_content = content.drop(['salary'], axis=1)
#目标Target
target = content['salary'].tolist()

X_content['merged'] = X_content.apply(lambda x: ''.join(str(x)), axis=1)
X_string = X_content['merged'].tolist()
print(len(X_string))

import jieba
def get_one_row_job_string(x_string_row):
    job_string = ''
    for i, element in enumerate(x_string_row.split('\n')):
        if len(element.split()) == 2:
            _, value = element.split()
            #i=0为id字段，不要
            if i==0:
                continue
            job_string +=value
        return job_string
import    re
def token(string):
    return re.findall('\w', string)

cutted_X = []
for i, row in enumerate(X_string):
    #print(row)
    #break
    job_string = get_one_row_job_string(row)
    cutted_X.append(' '.join(list(jieba.cut(''.join(token(job_string))))))
print(cutted_X[0])
print(len(cutted_X))

#使用TFIDF提取文本特征
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cutted_X)
#print(X[0])
      
#[10k-15k', '30k-50k', '40k-75k', '20k-40k', '20k-38k', '30k-50k', '40k-60k', '30k-50k', '40k-70k', '30k-50k']
print(target[:10])   
import numpy as np
#求平均值
target_numical = [np.mean(list(map(float,re.findall('\d+', s)))) for s in target]
Y = target_numical

#使用KNN模型
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=2)
model.fit(X, Y)
print(model.score)

def predicate_by_label(test_string, model):
      test_words = list(jieba.cut(test_string))
      test_vec = vectorizer.transform(test_words)
      predicate_value = model.predict(test_vec)
      return predicate_value[0]

test = '测试 北京 3年 专科'
print(test, predicate_by_label(test, model))
test = '测试 北京 4年 专科'
print(test, predicate_by_label(test, model))
test = '算法 北京 4年 本科'
print(test, predicate_by_label(test, model))
test = 'UI 北京 4年 本科'
print(test, predicate_by_label(test, model))
test = '广州Java本科3年掌握大数据'
print(test, predicate_by_label(test, model))
test = '沈阳Java硕士3年掌握大数据'
print(test, predicate_by_label(test, model))
test = '沈阳Java本科3年掌握大数据'
print(test, predicate_by_label(test, model))
test = '北京算法硕士3年掌握图像识别'
print(test, predicate_by_label(test, model))
'''
"广州Java本科3年掌握大数据",
"沈阳Java硕士3年掌握大数据", 
"沈阳Java本科3年掌握大数据", 
"北京算法硕士3年掌握图像识别"
'''

   id         positionName district stationname jobNature  \
0   3               财务产品经理      海淀区          上地        全职   
1   4                 产品专家      海淀区         中关村        全职   
2   5                 产品总监      海淀区         中关村        全职   
3   6  联盟广告产品运营专家/专员-【商业化】      海淀区          东单        全职   
4   7                 产品总监      海淀区         中关村        全职   

                   companyLabelList industryField   salary companySize  \
0  ['节日礼物', '年底双薪', '定期体检', '七险一金']    物流丨运输,软件开发  10k-15k   500-2000人   
1  ['五险一金', '弹性工作', '带薪年假', '免费两餐']          企业服务  30k-50k   500-2000人   
2  ['五险一金', '弹性工作', '带薪年假', '免费两餐']          企业服务  40k-75k   500-2000人   
3  ['股票期权', '弹性工作', '定期体检', '岗位晋升']         文娱丨内容  20k-40k     2000人以上   
4  ['领导好', '五险一金', '领军企业', '脑力密集型']      移动互联网,社交  20k-38k      15-50人   

                      skillLables           createTime companyFullName  \
0           ['产品设计', '后台', 'B2B']  2020-05-15 15:10:02  北京福佑多多信息技术有限公司   
1                              []  2020-

'\n"广州Java本科3年掌握大数据",\n"沈阳Java硕士3年掌握大数据", \n"沈阳Java本科3年掌握大数据", \n"北京算法硕士3年掌握图像识别"\n'